In [15]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")
pyproj.datadir.set_data_dir("D:/ProgramData/anaconda3/envs/crime-prediction/Library/share/proj")

In [16]:
pre_processed_file_path = 'data/preprocessed/preprocessed-5.csv'
household_income_data_file_path = 'data/preprocessed/household_income_data.geojson'

In [17]:
def parse_preprocessed_data():
    df = pd.read_csv(
                         pre_processed_file_path,
                         parse_dates=['Offense Date'],                       
                         dtype={
                             'Precinct': 'category',
                             'Offense Category': 'category',
                             'Latitude': np.float32,
                             'Longitude': np.float32
                         }
                        )
    df["Id"] = df.index
    gdf = gpd.GeoDataFrame(df, geometry= df["geometry"].apply(wkt.loads), crs='EPSG:4326')
    return gdf

In [18]:
crime_gdf = parse_preprocessed_data()
print(len(crime_gdf))

1280004


In [19]:
household_income_gdf = gpd.read_file(household_income_data_file_path)

In [20]:
print(household_income_gdf.columns)
print(crime_gdf.columns)

Index(['Tract Name', 'HouseHold Income', 'geometry'], dtype='object')
Index(['Report Number', 'Offense Date', 'Latitude', 'Longitude', 'Beat',
       'Precinct', 'Sector', 'Neighborhood', 'Reporting Area',
       'Offense Category', 'geometry', 'centroid', 'Temperature', 'Wind Speed',
       'Precipitation', 'Visibility', 'Tract Name', 'Tract Population',
       'Tract Population Below Poverty', 'White_Total', 'Black_Total',
       'Color_Total', '18_Under_Total', '65_Over_Total', 'Id',
       'StreetLight Count'],
      dtype='object')


In [21]:
crime_gdf = gpd.sjoin(crime_gdf, household_income_gdf, how="left", predicate="within") \
                .drop(columns=['index_right']) \
                .groupby("Id") \
                .first()
print(len(crime_gdf))

1280004


In [22]:
crime_gdf.to_csv('data/preprocessed/preprocessed-6.csv', index=False)